In [1]:
import os
import pandas as pd
from dotenv import load_dotenv, find_dotenv
import tensorflow as tf

load_dotenv(find_dotenv())
from crawl import Crawler

crawler = Crawler(api_key=os.getenv("API_KEY"))
# crawler.download_historical_data(category="crypto", symbol="BTC", location="./data")

In [2]:
# Reading data and old labels
new_data = pd.read_parquet("./data/BTCUSDT/")
old_labels = pd.read_csv("./data/labels.csv", index_col=0)

In [3]:
new_data

,OPEN,HIGH,LOW,CLOSE,VOLUME,QUOTE_ASSET_VOLUME,NUMBER_OF_TRADES,TAKER_BUY_BASE_ASSET_VOLUME,TAKER_BUY_QUOTE_ASSET_VOLUME,YEAR_AND_MONTH
OPEN_TIME,,,,,,,,,,
1502942400000,4261.48,4280.56,4261.48,4261.48,2.189061,9333.620962,9,0.489061,2089.104962,201708
1502942700000,4261.48,4261.48,4261.48,4261.48,0.000000,0.000000,0,0.000000,0.000000,201708
1502943000000,4261.48,4261.48,4261.48,4261.48,0.000000,0.000000,0,0.000000,0.000000,201708
1502943300000,4261.48,4264.88,4261.48,4261.48,0.484666,2065.447829,9,0.085455,364.218136,201708
1502943600000,4264.88,4266.29,4264.88,4266.29,2.328570,9931.161124,11,1.546491,6595.688039,201708
...,...,...,...,...,...,...,...,...,...,...
1690126800000,29898.00,29898.01,29894.40,29894.41,8.919730,266664.419999,778,2.553620,76340.981706,202307
1690127100000,29894.40,29899.57,29894.40,29899.57,19.828990,592819.512826,1080,12.367740,369756.001273,202307
1690127400000,29899.56,29899.57,29888.38,29888.39,18.528320,553900.170339,1181,2.602970,77815.346448,202307


In [4]:
btc_price = pd.DataFrame(index=pd.to_datetime(new_data.index[-5000:], unit="ms"))
btc_price["PRICE"] = (
    ((new_data["HIGH"] + new_data["LOW"] + new_data["CLOSE"]) / 3).tail(5000).values
)

In [5]:
# New training and test data set
from ulti import calculate_R_V, make_train_set, model_forecast
new_R_V = calculate_R_V(101, btc_price)
training_data = make_train_set(15, new_R_V[:-15], 1)

In [6]:
# Load the pre-trained model
tf.random.set_seed(1012)
model = tf.keras.models.load_model("./nq.h5")
# model.fit(training_data, epochs=5)

In [8]:
# Brownian motion
from sklearn.preprocessing import MinMaxScaler
import numpy as np
mu = 0.1
n = 500
dt = 1e-5
x0 = 1
sigma = 0.3
p_bm = np.exp((mu - sigma**2 / 2) * dt + sigma * np.random.normal(0, dt, n))
p_bm = MinMaxScaler().fit_transform(p_bm.reshape(-1, 1)).flatten()

In [9]:
predicton = model_forecast(model, new_R_V[-15:], 15).reshape(2)

1/1 [==============================] - 3s 3s/step


## Calculate raw labels and save

In [13]:
pred_labels = predicton[0] / (
    2 * np.sqrt(predicton[1] * np.random.choice(p_bm, size=1))
)
new_labels = np.vstack((old_labels.values, [pred_labels]))


In [106]:
pd.DataFrame(new_labels[-6000:-1]).to_csv("./data/labels.csv")

In [104]:
# Calculate scaled prediction
prediction = MinMaxScaler().fit_transform(new_labels)[-1][0]
submit_hour = (pd.Timestamp.now().ceil(freq="H") - pd.Timedelta("7H")).value // 10**6
print(submit_hour)

1690135200000


In [105]:
submission = pd.DataFrame(
    index=[0], data={"OPEN_TIME": submit_hour, "PREDICTION": prediction}
)
submission = submission.to_dict(orient="records", index=True)
submission

[{'OPEN_TIME': 1690135200000, 'PREDICTION': 0.46727589260202684}]

In [125]:
new_R_V[:-15].shape

(402, 2)

In [66]:
from submit import Submission

api_submit = Submission(api_key=os.getenv("API_KEY"))
api_submit.submit(False, submission, "BTCUSDT")

'1690123004422'

In [112]:
model.save("./nq.h5")

c:\Users\nguye\Downloads\NestQuant\.venv\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
